<a href="https://colab.research.google.com/github/MathieuDuboy/IncrementMp3Decibel/blob/main/increment_db.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Pré-requis : Installation de ffmpeg**

In [6]:
!apt-get install -y ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


**Importations**

In [7]:
from IPython import get_ipython
from IPython.display import display

import os
import zipfile
import shutil
import subprocess
import pandas as pd
from google.colab import files as colab_files

**Variables dynamiques à modifier si besoin**

In [8]:
# Augmentation du volume de XX Db
decibel = 1

# Dossier de contenus IN et OUT
input_folder = "/content/input_mp3"
output_folder = "/content/output_mp3"

# Enregistrer les résultats dans un fichier CSV
csv_path = "/content/audio_levels.csv"

# Compresser les fichiers modifiés en un nouveau .zip
output_zip = "/content/output_mp3.zip"


**Partie du code à jouer à chaque nouveau test**
1.   Télécharger le .zip contenant les .mp3
2.   Attendre la décompression
3.   Un .zip va se télécharger automatiquement ainsi que le .csv








In [11]:
# Télécharger le fichier .zip contenant les fichiers MP3
print("Téléchargez le fichier .zip contenant les fichiers MP3 :")
uploaded = colab_files.upload()

# Identifier le fichier ZIP téléchargé
input_zip = list(uploaded.keys())[0]

# Décompresser les fichiers MP3 dans un dossier temporaire
os.makedirs(input_folder, exist_ok=True)
os.makedirs(output_folder, exist_ok=True)

with zipfile.ZipFile(input_zip, 'r') as zip_ref:
    zip_ref.extractall(input_folder)

# Fonction pour récupérer le niveau sonore (RMS dB) avec FFmpeg
def get_audio_loudness(file_path):
    command = [
        "ffmpeg",
        "-i", file_path,
        "-filter:a", "volumedetect",
        "-f", "null",
        "-"
    ]
    result = subprocess.run(command, stderr=subprocess.PIPE, text=True)

    for line in result.stderr.split("\n"):
        if "mean_volume" in line:
            return float(line.split(":")[-1].strip().replace(" dB", ""))
    return None

# Stocker les résultats
data = []

# Parcours des fichiers MP3 pour augmenter le son
for filename in os.listdir(input_folder):
    if filename.endswith(".mp3"):
        input_path = os.path.join(input_folder, filename)
        output_filename = f"{os.path.splitext(filename)[0]}.mp3"
        output_path = os.path.join(output_folder, output_filename)

        # Niveau sonore initial
        initial_db = get_audio_loudness(input_path)

        # Commande FFmpeg pour augmenter le volume de XXdB tout en conservant les métadonnées
        command = [
            "ffmpeg",
            "-i", input_path,
            "-filter:a", "volume=" + str(decibel) + "dB",
            "-map_metadata", "0",  # Conserver les métadonnées d'origine
            output_path
        ]
        subprocess.run(command)

        # Niveau sonore après modification
        final_db = get_audio_loudness(output_path)

        # Ajouter les infos au tableau
        data.append([filename, initial_db, final_db])


df = pd.DataFrame(data, columns=["Nom du fichier", "dB avant", "dB après"])
df.to_csv(csv_path, index=False)


with zipfile.ZipFile(output_zip, 'w') as zipf:
    for root, dirs, files_in_dir in os.walk(output_folder):
        for file in files_in_dir:
            zipf.write(os.path.join(root, file), arcname=file)

# Télécharger le fichier .zip de sortie
print("Téléchargement du fichier .zip de sortie...")
colab_files.download(output_zip)

# Télécharger le fichier CSV des niveaux sonores
print("Téléchargement du fichier CSV avec les niveaux sonores avant/après...")
colab_files.download(csv_path)

Téléchargez le fichier .zip contenant les fichiers MP3 :


Saving output_mp3.zip to output_mp3.zip
Téléchargement du fichier .zip de sortie...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Téléchargement du fichier CSV avec les niveaux sonores avant/après...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>